In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [19]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen =ImageDataGenerator(rescale = 1)

In [20]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bbce69a0d9154fb98230bc8d3d675ca4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Gf8xmp6aXlCEloq3a4qr4wNCgjev-lhn7XA96qzOEAG0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_bbce69a0d9154fb98230bc8d3d675ca4.get_object(Bucket='fruittraining-donotdelete-pr-eq23lza8a7nqok', Key='Dataset Plant Disease.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [21]:
from io import BytesIO
import zipfile 
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [22]:
pwd

'/home/wsuser/work'

In [23]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/train',
                                            target_size = (128,128),batch_size = 32, class_mode = 'categorical')
x_test =  test_datagen.flow_from_directory('/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/test',
                                           target_size = (128,128),batch_size = 32, class_mode = 'categorical')

Found 3167 images belonging to 6 classes.
Found 1315 images belonging to 6 classes.


In [24]:
print(x_train.class_indices)

{'Apple___Black_rot': 0, 'Apple___healthy': 1, 'Corn_(maize)___Northern_Leaf_Blight': 2, 'Corn_(maize)___healthy': 3, 'Peach___Bacterial_spot': 4, 'Peach___healthy': 5}


In [25]:
model = Sequential()

In [26]:
model.add(Convolution2D(32,(3,3),input_shape = (128,128,3),activation = 'relu'))

In [27]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [28]:
model.add(Flatten())

In [29]:
model.add(Dense(units = 40 ,activation = 'relu'))
model.add(Dense(units = 20 ,activation = 'relu'))
model.add(Dense(units = 6,activation = 'softmax'))

In [30]:
model.compile(loss = 'categorical_crossentropy',optimizer = "adam",metrics = ["accuracy"])

In [31]:
model.fit_generator(x_train, steps_per_epoch = 70,epochs = 3,validation_data = x_test,validation_steps = 52)

/tmp/wsuser/ipykernel_156/1589208268.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train, steps_per_epoch = 70,epochs = 3,validation_data = x_test,validation_steps = 52)


Epoch 1/3
70/70 [==============================] - 27s 379ms/step - loss: 1.2681 - accuracy: 0.6038 - val_loss: 165.2823 - val_accuracy: 0.5605
Epoch 2/3
70/70 [==============================] - 22s 312ms/step - loss: 0.5382 - accuracy: 0.8155
Epoch 3/3
70/70 [==============================] - 23s 322ms/step - loss: 0.3825 - accuracy: 0.8665


In [32]:
model.save("fruit.h5")

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 40)                5080360   
                                                                 
 dense_1 (Dense)             (None, 20)                820       
                                                                 
 dense_2 (Dense)             (None, 6)                 126       
                                                        

In [34]:
!tar -zcvf Fruit-training.tgz fruit.h5

fruit.h5


In [35]:
ls

'Dataset Plant Disease'/        fruit-training.model.tgz
 Fruit                          Fruit-training-model.tgz
 fruit.h5                       Fruit.training.model.tgz
 fruit-training1.tgs            fruit-training.tgs
 fruit-training.model_new.tgs   Fruit-training.tgz
 fruit-training.model.tgs       Fruit.training.tgz


In [36]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 26.8 MB/s eta 0:00:01


In [42]:
from ibm_watson_machine_learning import APIClient
wml_credentials= {
                    "url" : "https://us-south.ml.cloud.ibm.com",
                    "apikey" : "tvhkSH6IXPhMpdA5ssNKYXWX-csA05S0edcy5IIFmldC"
                }
client = APIClient(wml_credentials)


In [43]:
client = APIClient(wml_credentials)

In [46]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [48]:
space_uid = guid_from_space_name(client,'Fruit Training')
print("space UID = " + space_uid)

space UID = 2f64d96c-70bb-4a35-b937-60cf1ff7ec17


In [49]:
client.set.default_space(space_uid)

'SUCCESS'

In [50]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [61]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid

'2b73a275-7cbf-420b-a912-eae7f436e0bc'

In [ ]:
model_details = client.repository.store_model(model= 'Fruit-training.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)


In [ ]:
model_id